In [1]:
import sys
import os
import muon as mu


# Add the Functions folder to your path
sys.path.append(os.path.abspath("/Users/alexandra/Desktop/EG-CLR/Functions"))

# Now import your functions
from QC_functions import *
from CLR_functions import *

In [3]:
# Load multimodal data
mdata = mu.read("/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/processed_data/multiome_data.h5mu")

adata_gem = mdata['rna']
adata_atac = mdata['atac']

/opt/anaconda3/envs/CLR/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/opt/anaconda3/envs/CLR/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [8]:
adata_gem

AnnData object with n_obs × n_vars = 10359 × 13703
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'S_score', 'G2M_score', 'phase'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'log1p'

In [9]:
# obtain orginal read counts
adata_org = sc.read_10x_mtx('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix',
                            gex_only = False)

gex_rows = list(map(lambda x: x == 'Gene Expression', adata_org.var['feature_types']))
atac_rows = list(map(lambda x: x == 'Peaks', adata_org.var['feature_types']))

adata_gem_org = adata_org[:, gex_rows].copy()
adata_atac_org = adata_org[:, atac_rows].copy()

adata_gem_org.var_names_make_unique()
adata_atac_org.var_names_make_unique()

In [10]:
# replace read counts with orgaianl read counts
gem_index = adata_gem.var_names
atac_index = adata_atac.var.index

cell_index = adata_gem.obs.index

adata_gem_new = adata_gem_org[:, adata_gem_org.var_names.isin(gem_index)].copy()
adata_atac_new = adata_atac_org[:, adata_atac_org.var_names.isin(atac_index)].copy()

adata_gem_new = adata_gem_new[adata_gem_new.obs_names.isin(cell_index)].copy()
adata_atac_new = adata_atac_new[adata_atac_new.obs_names.isin(cell_index)].copy()


In [11]:
# normalizing scRNA-seq by chromasomex 
adata_gem_new = assign_chr('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix/features.tsv.gz', adata_gem_new)
adata_gem_chrx = subset_adata_by_chromosome(adata_gem_new,-1)
sc.pp.normalize_total(adata_gem_chrx, target_sum=1e4)
sc.pp.log1p(adata_gem_chrx)
adata_gem_chrx

AnnData object with n_obs × n_vars = 10359 × 503
    var: 'gene_ids', 'feature_types', 'Chromosome', 'Start', 'End'
    uns: 'log1p'

In [12]:
# define promoter, enhancer and gene body regions
adata_atac_new = assign_loc('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix/features.tsv.gz',
    "/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/atac_peak_annotation.tsv", adata_atac_new)
adata_atac_new = assign_chr('/Volumes/G-DRIVE mobile USB-C/Single-cell_data/K562/10x/ISSAACC-seq_generated/hg19_10xCloud_aligned_data/filtered_feature_bc_matrix/features.tsv.gz', adata_atac_new)
adata_atac_chrx = subset_adata_by_chromosome(adata_atac_new,-1)
adata_atac_chrx

AnnData object with n_obs × n_vars = 10359 × 3424
    var: 'gene_ids', 'feature_types', 'gene', 'distance', 'peak_type', 'Chromosome', 'Start', 'End'

In [13]:
# binarize scATAC-seq 
adata_atac_chrx.X = (adata_atac_chrx.X > 0).astype(int)

In [56]:
adata_atac_chrx_50 = adata_atac_chrx[:1000,:2989].copy()
adata_gem_chrx_50 = adata_gem_chrx[:1000,:50].copy()

In [57]:
adata_atac_chrx_50

AnnData object with n_obs × n_vars = 1000 × 2989
    var: 'gene_ids', 'feature_types', 'gene', 'distance', 'peak_type', 'Chromosome', 'Start', 'End'

In [58]:
MI_Matrix_chrx_100 = MI_Matrix_MIinfoClassif(adata_gem_chrx_50, adata_atac_chrx_50)

In [ ]:
# compute CLR matrix for GATA1 -> ChrX
MI_Matrix_chrx = MI_Matrix_MIinfoClassif(adata_gem_chrx, adata_atac_chrx)

In [ ]:
CLR_matrix_chrx = CLR_Matrix(MI_Matrix_chrx)

In [ ]:
write_matrixes("ChrX_allgene_allpeaks",GATA1)

In [ ]:
adata_CLR_matrix_chrx = ad.AnnData(X=CLR_matrix_chrx.values)
adata_CLR_matrix_chrx.obs = adata_atac_chrx.var.copy()
adata_CLR_matrix_chrx.var = adata_gem_chrx.var.copy()
adata_CLR_matrix_chrx

In [ ]:
CLR_matrix_chrx = pd.DataFrame(CLR_matrix_chrx.values, index=adata_atac_chrx.var['gene'], columns=adata_gem_chrx.var['geng_ids'])
GATA1 = CLR_matrix_chrx["GATA1"].copy()
GATA1

In [ ]:
# This function takes a gene name and a CLR matrix, and returns a DataFrame containing the CLR values for the specified genes.
def EG_pair_by_name(gene, CLR_Matrix):
    
    if gene not in CLR_Matrix.columns:
        print(f"Warning: {gene} not found in CLR matrix.")
    else:
        EG_pair = CLR_Matrix[gene].copy()
        EG_pair = EG_pair[EG_pair > 0]
        #GATA1_EG_pair = GATA1_EG_pair.reset_index()
        
        
        #GATA1_EG_pair.columns = ['Peak', 'CLR_value']
        GATA1_EG_pair = EG_pair.index.str.extract(r'^(chr\w+):(\d+)-(\d+)$')
        GATA1_EG_pair.columns = ['chr', 'start', 'end']

    
        return GATA1_EG_pair

In [ ]:
GATA1_EG_pair = EG_pair_by_name("GATA1", CLR_matrix_chrx)
GATA1_EG_pair

In [ ]:
CRISPRi_data = pd.read_excel('/Users/alexandra/Desktop/Data/CRISPRiFlowFISH/41588_2019_538_MOESM3_ESM.xlsx',
                             sheet_name='Supplementary Table 6a', 
                             skiprows=0,
                             header=1)
CRISPRi_GATA1 = CRISPRi_data[CRISPRi_data['Gene'] == 'GATA1'].copy()
CRISPRi_GATA1 = CRISPRi_GATA1[['chr', 'start', 'end', 'class', 'Significant']].copy()
CRISPRi_GATA1

In [ ]:
CRISPRi_GATA1_TP = CRISPRi_GATA1[CRISPRi_GATA1["Significant"] == True]
CRISPRi_GATA1_TN = CRISPRi_GATA1[CRISPRi_GATA1["Significant"] == False]

In [ ]:
CRISPRi_GATA1_TP = CRISPRi_GATA1_TP[['chr', 'start', 'end']].copy()
CRISPRi_GATA1_TP

In [ ]:
CRISPRi_GATA1_TN

In [ ]:
CRISPRi_GATA1_start =  CRISPRi_GATA1['start'].min()
CRISPRi_GATA1_end = CRISPRi_GATA1['end'].max() 

print("CRISPR for GATA1 starts at ",CRISPRi_GATA1_start, " end at ", CRISPRi_GATA1_end)

In [ ]:
# Filter for overlapping peaks
GATA1_EG_pair_limited = GATA1_EG_pair[
    (GATA1_EG_pair["end"].values.astype(int) < CRISPRi_GATA1_end ) & 
    (GATA1_EG_pair["start"].values.astype(int) > CRISPRi_GATA1_start)
]
GATA1_EG_pair_limited

In [ ]:
GATA1_EG_pair_limited

In [ ]:

CRISPRi_GATA1_TP

In [ ]:
import pandas as pd
import pybedtools

# Example DataFrame
df = pd.DataFrame({
    "chr": ["chr1", "chr1", "chr2"],
    "start": [100, 200, 300],
    "end": [150, 250, 350]
})

# Convert to BedTool
bed_str = df.to_csv(sep="\t", header=False, index=False)
bed = pybedtools.BedTool(bed_str, from_string=True)

# Now you can use .intersect(), .saveas(), etc.
print(bed)

In [ ]:
bed_str

In [ ]:
# Load as BedTool
GATA1_EG_pair_limited_bed_str = GATA1_EG_pair_limited.to_csv(sep="\t", header=False, index=False)
CRISPRi_GATA1_TP_bed_str = CRISPRi_GATA1_TP.to_csv(sep="\t", header=False, index=False)

GATA1_EG_pair_limited_bed = pybedtools.BedTool(GATA1_EG_pair_limited_bed_str, from_string=True)
CRISPRi_GATA1_TP_bed = pybedtools.BedTool(CRISPRi_GATA1_TP_bed_str, from_string=True)

# Intersect: return entries in A that overlap with B
overlap = GATA1_EG_pair_limited_bed.intersect(CRISPRi_GATA1_TP_bed, u=True)  # 'u=True' gives only entries from A

# Convert result to DataFrame
overlap_df = overlap.to_dataframe(names=["chr", "start", "end"])
overlap_df


# remove peaks that does not exist at the begining